<div align="center"><h1>Potential Flow Model</h1></div>
<div align="center"><h2>log-reduction calculator</h2></div>

In [1]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib
import ipywidgets as wd
from os import system
import os
from matplotlib.gridspec import GridSpec 

#import jupypft.attachmentRateCFT as CFT
#import jupypft.plotBTC as BTC

''' GLOBAL CONSTANTS '''
PI = 3.141592
I = 10**np.linspace(-5,0,num=100)

''' FUNCTION CALCULATORS'''
def calculateAll(var_dict):
    K = var_dict['K']
    THETA = var_dict['n']
    Qin = var_dict['Qin']/86400
    f = var_dict['Qout']/var_dict['Qin']
    H = var_dict['H']
    r = var_dict['rs']
    C0 = 1.0
    decayRate = var_dict['mu']/86400
    attchRate = var_dict['katt']/86400
    delY,delZ = 1.35,var_dict['H']
        
    def flowNumber(): return (4.0*K*I*H*r) / (Qin*(1+f))
    def uChar(): return -(K*I*flowNumber())/(THETA*(flowNumber() + 1))
    def tChar(): return -r/uChar()
    def cDecay(): return C0 * np.exp(-decayRate * tChar())
    def cAttach(): return C0 * np.exp(-attchRate * tChar())
    def cDilut(): return (C0 * Qin) / (-uChar() * delY * delZ * THETA)
    def cBoth(): return (C0 * Qin) / (-uChar() * delY * delZ * THETA) * np.exp(-decayRate * tChar())
    def cTrice(): return (C0 * Qin) / (-uChar() * delY * delZ * THETA) * np.exp(-(decayRate+attchRate) * tChar())
    def findSweet(): return np.argmin(np.abs(cTrice() - np.max(cTrice())))  
    
    cDec = cDecay()
    cDil = cDilut()
    cAtt = cAttach()
    cBot = cBoth()
    cAll = cTrice()
    i = findSweet()
    worstC = cAll[i]
    worstI = I[i]
    
    results_dict = dict(cDec=cDec,cDil=cDil,cAtt=cAtt,cBot=cBot,cAll=cAll,i=i,worstC=worstC,worstI=worstI)
    return results_dict

In [2]:
gs_Parameters = wd.GridspecLayout(8,2)

var_keys = ["K","H","rs","Qin","Qout","n","mu","katt"]
init_values = [0.01,20,40,0.24,2.4,0.35,0.30,0.10]
min_values = [1.0E-6,0.1,0.1,0.001,0.001,0.01,1.0E-6,1.0E-6]
max_values = [1.0E+2,1000,1000,1000,1000,0.999,1.0E+6,1.0E+6]

var_dict = {k:v for k,v in zip(var_keys,init_values)}

lbl_Parameters = [wd.Label(txt) for txt in 
                      ["Hydraulic conductivity [m/d]",
                      "Aquifer thickness [m]",
                      "Setback distance [m]",
                      "Contaminant influx [m3/d]",
                      "Extraction flow rate [m3/d]",
                      "Porosity [-]",
                      "Inactivation rate [1/d]",
                      "Attachment rate [1/d]"]]

val_Parameters = [wd.BoundedFloatText(value=v,min=minv,max=maxv,description=txt)
                  for v,minv,maxv,txt in 
                  zip(init_values,min_values,max_values,
                         [r"$K$",
                          r"$H$",
                          r"$r_s$",
                          r"$Q_{\rm in}$",
                          r"$Q_{\rm out}$",
                          r"$n$",
                          r"$\mu_m$",
                          r"$k_{\rm att}$"])]

for i,(lbl,val) in enumerate(zip(lbl_Parameters,val_Parameters)):
    gs_Parameters[i,0],gs_Parameters[i,1] = lbl,val

def update_values(_):
    '''Updates var_dict with the widgets values'''
    for k,v in zip(var_dict.keys(),val_Parameters):
        var_dict[k] = v.value    

for val_Widget in val_Parameters:
    val_Widget.observe(update_values,'value')

update_values(None)

In [3]:
layout = wd.Layout(justify_content='center',align_items='center')
output = wd.Output(layout=layout)

bbox = dict(boxstyle='round', facecolor='mintcream', alpha=0.90)
arrowprops = dict(arrowstyle="->",connectionstyle="angle,angleA=90,angleB=40,rad=5")
fontdict = dict(size=12)

@output.capture(clear_output=True)
def plot_PFM(_):
    
    update_values(_)
    cDec,cDil,cAtt,cBot,cAll,i,worstC,worstI = calculateAll(var_dict).values()
    
    with np.errstate(divide='ignore'):
        logs_cDec = -np.log10(cDec)
        logs_cDil = -np.log10(cDil)
        logs_cAtt = -np.log10(cAtt)
        logs_cAll = -np.log10(cAll)
    
    
    fig,ax = plt.subplots(figsize=(8,6))

    #### log-removals
    ax.plot(I,logs_cDec,label="Due decay",lw=3,ls="dashed",alpha=0.8)
    ax.plot(I,logs_cDil,label="Due dilution",lw=3,ls="dashed",alpha=0.8)
    ax.plot(I,logs_cAtt,label="Due attachment",lw=2,ls="dashed",alpha=0.6)
    ax.plot(I,logs_cAll,label="Combined effect",lw=3,c='k',alpha=0.9)

    ####################################<
    #Line worst case scenario
    #ax.axvline(x=I[i], lw=1, ls="dashed", c="red",alpha=0.5)
    ax.axhline(y=4, lw=1, ls='dashed', c='r', alpha = 0.5)
    
    annotation = \
        r"$\bf{-\log(C/C_0)} = $" + "{:.1f}".format(-np.log10(worstC)) + \
        "\n@" + r" $\bf{I} = $" + "{:.1E}".format(worstI)
    
    ax.scatter([worstI],[-np.log10(worstC)], 
               c='r', marker='X', s=250, zorder=2,
               label="\n\nWorst case:\n" + annotation)

    #####################################
    ax.set(xscale="log",xlim=(1.0E-4,0.5),ylim=(-0.1,10))
    ax.set_xlabel("Water table gradient\n$I$ [m/m]",fontdict=fontdict)
    ax.set_ylabel("log-reductions\n$-\log(C/C_0)$ [-]",fontdict=fontdict)
    ax.legend(loc="center left",bbox_to_anchor=(1.02,0.5),shadow=True)
   
    plt.show()

In [4]:
plotButton = wd.Button(description="Plot!",icon="fa-carrot")
plotButton.on_click(plot_PFM)

mainBox = wd.VBox([gs_Parameters,plotButton,output],layout=layout)

In [5]:
mainBox